In [1]:
import ast
from typing import Dict, Set, List, Any
import json
from pathlib import Path

class CallGraphBuilder(ast.NodeVisitor):
    def __init__(self):
        # 存储所有的函数定义
        self.functions: Set[str] = set()
        # 存储调用关系 {caller: {callee1, callee2, ...}}
        self.calls: Dict[str, Set[str]] = {}
        # 当前正在访问的函数名
        self.current_function: str = None
        
    def visit_FunctionDef(self, node: ast.FunctionDef):
        """访问函数定义"""
        function_name = node.name
        self.functions.add(function_name)
        
        # 保存上一个函数上下文
        previous_function = self.current_function
        self.current_function = function_name
        
        # 初始化该函数的调用集合
        if function_name not in self.calls:
            self.calls[function_name] = set()
        
        # 访问函数体
        self.generic_visit(node)
        
        # 恢复上一个函数上下文
        self.current_function = previous_function
        
    def visit_Call(self, node: ast.Call):
        """访问函数调用"""
        if isinstance(node.func, ast.Name):
            # 直接调用函数名的情况
            callee = node.func.id
            if self.current_function:  # 如果在函数内部
                self.calls[self.current_function].add(callee)
        elif isinstance(node.func, ast.Attribute):
            # 处理方法调用 (obj.method())
            callee = node.func.attr
            if self.current_function:
                self.calls[self.current_function].add(callee)
        
        # 继续访问调用的参数
        self.generic_visit(node)

def build_callgraph(source_code: str) -> Dict[str, Any]:
    """构建调用图并返回JSON格式的数据"""
    # 解析源代码
    tree = ast.parse(source_code)
    
    # 构建调用图
    builder = CallGraphBuilder()
    builder.visit(tree)
    
    # 构建节点列表
    nodes = [{"id": func_name, "type": "function"} for func_name in builder.functions]
    
    # 构建边列表
    edges = []
    for caller, callees in builder.calls.items():
        for callee in callees:
            edges.append({
                "source": caller,
                "target": callee,
                "type": "call"
            })
    
    return {
        "nodes": nodes,
        "edges": edges
    }

def process_python_file(file_path: Path):
    """处理单个Python文件，生成调用图JSON"""
    try:
        # 读取文件内容
        with open(file_path, 'r', encoding='utf-8') as f:
            source_code = f.read()
        
        # 构建调用图
        callgraph_json = build_callgraph(source_code)
        
        # 创建输出目录
        output_dir = Path('callgraph_output')
        output_dir.mkdir(exist_ok=True)
        
        # 保存JSON文件
        json_path = output_dir / f"{file_path.stem}_callgraph.json"
        with open(json_path, 'w', encoding='utf-8') as f:
            json.dump(callgraph_json, f, indent=2, ensure_ascii=False)
        
        print(f"已生成调用图: {json_path}")
        return True
        
    except Exception as e:
        print(f"处理文件 {file_path} 时出错: {str(e)}")
        return False

def process_directory(directory: str):
    """遍历目录处理所有Python文件"""
    directory = Path(directory)
    
    if not directory.exists():
        raise ValueError(f"目录不存在: {directory}")
    
    for file_path in directory.rglob("*.py"):
        print(f"\n处理文件: {file_path}")
        process_python_file(file_path)

if __name__ == "__main__":
    data_dir = "./data"
    process_directory(data_dir)



处理文件: data/sample.py
已生成调用图: callgraph_output/sample_callgraph.json

处理文件: data/ideogram.py
已生成调用图: callgraph_output/ideogram_callgraph.json

处理文件: data/embeddings.py
已生成调用图: callgraph_output/embeddings_callgraph.json

处理文件: data/block_cost_config.py
已生成调用图: callgraph_output/block_cost_config_callgraph.json

处理文件: data/manager.py
已生成调用图: callgraph_output/manager_callgraph.json

处理文件: data/sum_of_subsets.py
已生成调用图: callgraph_output/sum_of_subsets_callgraph.json

处理文件: data/fact_checker.py
已生成调用图: callgraph_output/fact_checker_callgraph.json

处理文件: data/credit.py
已生成调用图: callgraph_output/credit_callgraph.json

处理文件: data/agent.py
已生成调用图: callgraph_output/agent_callgraph.json

处理文件: data/cache.py
已生成调用图: callgraph_output/cache_callgraph.json

处理文件: data/check_actions_status.py
已生成调用图: callgraph_output/check_actions_status_callgraph.json

处理文件: data/decorator.py
已生成调用图: callgraph_output/decorator_callgraph.json

处理文件: data/repo.py
已生成调用图: callgraph_output/repo_callgraph.json

处理文件: data/t